# Atrial Fibrillation Features

As with activity classification, we will featurize our time series signal and throw those features into a classifier. For AF we will build a two-class classifier using the inter-beat-interval time series. This time series can be derived by taking the difference between successive QRS complex locations as provided by the Pan-Tompkins algorithm in the previous videos. We source our features from a couple submissions to the [Computing in Cardiology Challenge 2017](https://physionet.org/content/challenge-2017/1.0.0/). 

>[Behar, Rosenberg, Yaniv, Oster. Rhythm and Quality Classification from Short ECGs Recorded Using a Mobile Device. Computing in Cardiology Challenge 2017.](http://www.cinc.org/archives/2017/pdf/165-056.pdf)  

and   
>[Bonizzi, Driessens, Karel. Detection of Atrial Fibrillation Episodes from Short Single Lead Recordings by Means of Ensemble Learning. Computing in Cardiology Challenge 2017.](http://www.cinc.org/archives/2017/pdf/169-313.pdf)


Recall that the RR interval time series is irregularly sampled because we get a new measurement at each heart beat, instead of at some fixed interval in time. Many of the features we want can be computed on an irregular time series, but some, especially frequency domain features, cannot. In that case, we will first have to make this uniform by using interpolation.


## Imports

In [33]:
import glob
import os

import numpy as np
import pandas as pd

## Load Data

In [34]:
fs = 300
data_dir = '/data/cinc/'
ref = pd.read_csv(data_dir + 'REFERENCE.csv')
ref = dict(zip(ref.record, ref.rhythm))
base = lambda f: os.path.splitext(os.path.basename(f))[0]
files = sorted(glob.glob(data_dir + '*.npz'))
qrs = []
labels = []
for f in files:
    with np.load(f) as npz:
        qrs.append(npz['qrs'])
    labels.append(ref[base(f)])

In [35]:
qrs

[array([ 117,  331,  549,  786, 1029, 1260, 1499, 1743, 1985, 2219, 2459,
        2702, 2941, 3178, 3422, 3668, 3903, 4129, 4359, 4588, 4816, 5034,
        5249, 5474, 5700, 5932, 6153, 6374, 6596, 6816, 7029, 7248, 7471,
        7686, 7898, 8117, 8345, 8575, 8809]),
 array([ 153,  459,  760, 1065, 1383, 1688, 2003, 2324, 2632, 2741, 2953,
        3278, 3585, 3895, 4208, 4511, 4807, 5109, 5359, 5463, 5559, 5619,
        5762, 5838, 5939, 6055, 6230, 6520, 6807, 7104, 7414, 7518, 7682,
        7980, 8275, 8560, 8859]),
 array([  268,   492,   724,   956,  1192,  1429,  1654,  1868,  2080,
         2297,  2521,  2752,  2997,  3246,  3492,  3722,  3943,  4160,
         4380,  4602,  4829,  5061,  5297,  5531,  5764,  5988,  6206,
         6417,  6628,  6843,  7065,  7301,  7544,  7786,  8020,  8239,
         8452,  8662,  8877,  9097,  9325,  9559,  9794, 10024, 10242,
        10457, 10663, 10873, 11087, 11298, 11504, 11706, 11915, 12077,
        12142, 12377, 12610, 12840, 13068, 13293, 

In [36]:
for i in qrs:
    if len(i)==0:
        print('is there') 

is there


In [37]:
len(qrs)

7416

In [38]:
for subarray in qrs:
    if len(subarray)==0:
        print(qrs.index(subarray))

4682


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
qrs[4682]

array([], dtype=int64)

In [40]:
type(qrs)

list

In [41]:
qrs.pop(4682)

array([], dtype=int64)

In [42]:
len(qrs)

7415

In [44]:
qrs[4682]

array([ 248,  442,  523,  787, 1052, 1242, 1501, 1767, 2036, 2307, 2579,
       2773, 3029, 3304, 3581, 3842, 4110, 4385, 4659, 4855, 5112, 5377,
       5638, 5912, 6187, 6462, 6735, 6928, 7166, 7431, 7615, 7856, 8125,
       8396, 8668, 8949])

In [45]:
qrs[0]

array([ 117,  331,  549,  786, 1029, 1260, 1499, 1743, 1985, 2219, 2459,
       2702, 2941, 3178, 3422, 3668, 3903, 4129, 4359, 4588, 4816, 5034,
       5249, 5474, 5700, 5932, 6153, 6374, 6596, 6816, 7029, 7248, 7471,
       7686, 7898, 8117, 8345, 8575, 8809])

In [46]:
len(qrs[0])

39

In [47]:
len(qrs[1])

37

In [51]:
len(qrs[7414])

31

In [52]:
np.shape(qrs)

(7415,)

In [53]:
len(qrs)

7415

In [54]:
len(labels)

7416

## Features

The features for the AF detection algorithm are computed from the RR interval time series. We use the time domain and frequency domain features listed below.

### Time domain
 - minimum RR interval
 - maximum RR interval
 - median RR interval
 - average RR interval
 - standard deviation of RR intervals
 - number of RR interval outliers
   - An RR interval is an outlier if it is greater than 1.2x the average RR interval in the ECG record

 - root-mean-square of the difference between adjacent RR intervals
 - percent of RR interval differences greater than 50 milliseconds

### Frequency domain
The RR interval time series is not sampled regularly in time. We only have a datapoint every heart beat. Before we can compute any frequency domain features, the time series must be resampled so that we get uniform data points. Resample the RR interval time series to 4 Hz before computing the features below.

 - peak magnitude between 0.04 Hz and 0.15 Hz in the regularized RR interval time series
 - main frequency between 0.04 Hz and 0.15 Hz in the regularized RR interval time series
 - peak magnitude between 0.15 Hz and 0.4 Hz in the regularized RR interval time series
 - main frequency between 0.15 Hz and 0.4 Hz in the regularized RR interval time series

In [55]:
a=np.diff(qrs[0])

In [56]:
np.min(a)

212

In [69]:

def Featurize(qrs_inds, fs):
        """Featurize the qrs complex locations time series.

        Args:
            qrs_inds: (np.array of number) the sample indices of the QRS complex locations
            fs: (number) the sampling rate

        Returns:
            n-tuple of features
        """
        # Compute the RR interval time series
        rr=np.diff(qrs_inds)

        # Compute time domain features
        min_rr = np.min(rr)
        max_rr = np.max(rr)
        median_rr = np.median(rr)
        mean_rr = np.mean(rr)
        std_rr = np.std(rr)
        n_outliers = np.sum(rr>1.2*np.mean(rr))
        rmssd = np.sqrt(np.mean(np.diff(rr)))
        pdrr_50 = np.mean(np.diff(rr)/fs>0.05)

        # Regularly resample the RR interval time series
        rr_ts=np.arange(qrs_inds[0]/fs,qrs_inds[-2]/fs,1/4)
        rr_interp=np.interp( rr_ts,qrs_inds[:-1]/fs,rr)

        # Compute the Fourier transform of the regular RR interval time series
        freq=np.fft.rfftfreq(len(rr_interp),1/4)
        fft_mag=np.abs(np.fft.rfft(rr_interp))

        # Compute frequency domain features
        lf_mag = np.max(fft_mag[(freq>=0.04)&(freq<=0.15)])
        lf_freq = freq[np.argmax(fft_mag[(freq>=0.04)&(freq<=0.15)])]
        hf_mag = np.max(fft_mag[(freq>=0.15)&(freq<=0.4)])
        hf_freq = freq[np.argmax(fft_mag[(freq>=0.15)&(freq<=0.4)])]

        return (min_rr, max_rr, median_rr, n_outliers, mean_rr, std_rr, rmssd, pdrr_50,
            lf_mag, lf_freq, hf_mag, hf_freq)
#         return (min_rr, max_rr, median_rr, n_outliers, mean_rr, std_rr,pdrr_50,
#             lf_mag, lf_freq, hf_mag, hf_freq)


In [ ]:
feats=[Featurize(qrs_inds,fs) for qrs_inds in qrs]
X,y=np.array(feats),np.array(labels)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in sqrt
